- get chatgpt to accept prompts
- get chatgpt to output prompts

In [3]:
!pip install cohere
!pip install tiktoken
!pip install openai

  Using cached cohere-4.35-py3-none-any.whl (48 kB)
  Using cached backoff-2.2.1-py3-none-any.whl (15 kB)
  Using cached fastavro-1.8.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.7 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires openai, which is not installed.
  Using cached openai-1.3.3-py3-none-any.whl (220 kB)
  Using cached httpx-0.25.1-py3-none-any.whl (75 kB)
  Using cached httpcore-1.0.2-py3-none-any.whl (76 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)


In [18]:
import os
from openai import OpenAI
from google.colab import userdata
import pandas as pd
import re
from datetime import datetime, timedelta
client = OpenAI(api_key=userdata.get('API_KEY'))
MODEL = "gpt-3.5-turbo-16k"


In [5]:
# #Initial call to GPT to understand the trials
# init_sys_call = [
#     {"role": "system",
#     "content": """Your task is to give judgement about where you were based on the time we give. You will give 4 alternative options. You MUST select one.

#                   Example:

#                   Presented:
#                   Location 1 at 6PM 11/08/2019
#                   Location 2 at 7PM 12/08/2019
#                   ...

#                   Question:
#                   Where were you at 6PM 11/09/2019:
#                   A. Location 2
#                   B. Location 3
#                   C. Location 1
#                   D. Location 30

#                   Response:
#                   C

#                   Where were you at 7PM 12/09/2019:
#                   A. Location 2
#                   B. Location 3
#                   C. Location 9
#                   D. Location 30

#                   Response:
#                   A
#                   ..."""
#               }]

In [28]:
events = pd.read_csv(userdata.get('EVENTS_FILE'))
experiments = pd.read_csv(userdata.get('EXPERIMENTS_FILE'))

In [29]:
# Helpful constants
EVENT_DF = 0
EVENT_OFFSET = 1
EXPERIMENT_DF = 2
EXPERIMENT_OFFSET = 3


def create_event_experiment_dfs(uuid):
  """
  create_event_experiment_dfs() creates the appropriate dataframes and index offsets to reference events and experiments.

  :param USER_ID: the user's unique identification string
  :return: user's events, event offset index, user's experiment, experiment offset index
  """

  events_temp = events[['ID', 'StartDateTime Local', 'GPS Cluster Original']][events['USER ID']==uuid]
  experiments_temp = experiments[['ID', 'Target', 'A', 'B', 'C', 'D']][experiments['USER ID']==uuid]

  # offsets
  events_offset = events_temp.index.min()
  experiments_offset = experiments_temp.index.min()
  return (events_temp, events_offset, experiments_temp, experiments_offset)

temp = create_event_experiment_dfs("ap-northeast-1:e7c916fc-736e-47e6-979a-c1c937ebe094")


In [30]:
# Let GPT know what to do and where it was

def gen_location_and_instruct(Event_Exp_df):
  """
  gen_location_and_instruct() turns each entry of event_df as a context for GPT and also
  adds the initial context to allow the model to understand what it is to do

  :param Event_Exp_df: the list of the event_df, event_offset, exp_df, exp_offset
  :return: the structured msg containing where a particular participant was located
  """

  events_df = Event_Exp_df[EVENT_DF]
  events_offset = Event_Exp_df[EVENT_OFFSET]
  location_data = []

  # Each entry is a context for GPT
  for i in range(len(events_df)):
    curr_datetime = events_df.loc[i+events_offset, "StartDateTime Local"]
    # date_pattern = r"(\d{4}-\d{2}-\d{2})T"
    # time_pattern = r"\d{4}-\d{2}-\d{2}T(\d{2}:\d{2}:\d{2})Z"
    # date = re.search(date_pattern, datetime).group(1)
    # time = re.search(time_pattern, datetime).group(1)

    datetime_obj = datetime.fromisoformat(curr_datetime.rstrip("Z"))
    date = datetime_obj.strftime("%d %B")
    day = datetime_obj.strftime("%A")
    time = datetime_obj.strftime("%I:%M %p")


    gps_cluster = events_df.loc[i+events_offset, "GPS Cluster Original"]
    location_data.append({
        "role": "user",
        "content": f"On {day}, {date} at {time} you were at location {gps_cluster}"
        # On Thursday, the 25th of May 2019 at 10 am you were at location 16
        #Location {gps_cluster} at {time}, {date}"
    })

  # variables are holding onto the latest dates, it's known that questioning took place a week after
  new_datetime_obj = datetime_obj + timedelta(days=7)
  date = new_datetime_obj.strftime("%d %B")
  day = new_datetime_obj.strftime("%A")

  location_data.insert(0,
   {"role": "system",
    "content": """Today is {}, {}. Your task is to give judgement about where you were based on the time we give. You will give 4 alternative options. You MUST select one.

                  Example:

                  Presented:
                  Location 1 at 6PM 11/08/2019
                  Location 2 at 7PM 12/08/2019
                  ...

                  Question:
                  Where were you at 6PM 11/09/2019:
                  A. Location 2
                  B. Location 3
                  C. Location 1
                  D. Location 30

                  Response:
                  C

                  Where were you at 7PM 12/09/2019:
                  A. Location 2
                  B. Location 3
                  C. Location 9
                  D. Location 30

                  Response:
                  A
                  ...""".format(day, date)
    })



  return location_data

  # 2019-08-11T06:00:00Z
  #  print(events_1.loc[i, "ID"], events_1.loc[i, "GPS Cluster Original"])
temp_loc = gen_location_and_instruct(temp)
temp_loc

[{'role': 'system',
  'content': 'Today is Saturday, 07 September. Your task is to give judgement about where you were based on the time we give. You will give 4 alternative options. You MUST select one.\n\n                  Example:\n\n                  Presented:\n                  Location 1 at 6PM 11/08/2019\n                  Location 2 at 7PM 12/08/2019\n                  ...\n\n                  Question:\n                  Where were you at 6PM 11/09/2019:\n                  A. Location 2\n                  B. Location 3\n                  C. Location 1\n                  D. Location 30\n\n                  Response:\n                  C\n\n                  Where were you at 7PM 12/09/2019:\n                  A. Location 2\n                  B. Location 3\n                  C. Location 9\n                  D. Location 30\n\n                  Response:\n                  A\n                  ...'},
 {'role': 'user',
  'content': 'On Sunday, 11 August at 01:00 AM you were at loc

In [31]:
# Form Questions for GPT

def gen_questions(Event_Exp_df):
  """
  gen_questions() generates the questions that will be fed to GPT by using the
  experiment_df

  :param Event_Exp_df: the list of the event_df, event_offset, exp_df, exp_offset
  :return: the structured msg containing questions from the experiment for a
           given participant
  """

  events_df = Event_Exp_df[EVENT_DF]
  experiments_df = Event_Exp_df[EXPERIMENT_DF]
  experiments_offset = Event_Exp_df[EXPERIMENT_OFFSET]
  question_data = []

  # Turn each entry into an input for GPT
  for i in range(len(experiments_df)):
    question = ""
    response_id = experiments_df.loc[i+experiments_offset, "Target"]
    correct_event_id = experiments_df.loc[i+experiments_offset, response_id]

    # Perform lookup for experiments_df from events_df
    cluster_question = []
    for alpha in ['A', 'B', 'C', 'D']:
      alpha_response_id = experiments_df.loc[i+experiments_offset, alpha]
      cluster_question.append(events_df.loc[alpha_response_id, "GPS Cluster Original"])

    # Datetime processing
    datetime = events_df.loc[correct_event_id, "StartDateTime Local"]
    date_pattern = r"(\d{4}-\d{2}-\d{2})T"
    time_pattern = r"\d{4}-\d{2}-\d{2}T(\d{2}:\d{2}:\d{2})Z"
    date = re.search(date_pattern, datetime).group(1)
    time = re.search(time_pattern, datetime).group(1)

    # Question forming
    question += f"Where were you at {time} {date}:\n"
    question += f"A. {cluster_question[0]}\n"
    question += f"B. {cluster_question[1]}\n"
    question += f"C. {cluster_question[2]}\n"
    question += f"D. {cluster_question[3]}"

    question_data.append({
        "role": "user",
        "content": question
    })

  return question_data

temp_ques = gen_questions(temp)

In [32]:
# Pricing (per 1k token) is specific for this model: GPT 3.5 Turbo (16k)

def run_gpt(Event_Exp_df, location_data, question_data):
  """
  run_gpt() is the main code for interacting with the OpenAI API to obtain expected output

  :param Event_Exp_df: the list of the event_df, event_offset, exp_df, exp_offset
  :param location_data: the structured location information for GPT
  :param question_data: the structured questions for GPT
  :return: an array of output as specified by the experiment
  """
  gpt_pricing = {"input": 0.001, "output": 0.002}
  experiments_df = Event_Exp_df[EXPERIMENT_DF]
  output_msg = []
  message = {}
  cost = 0

  for i in range(len(experiments_df)):
    message = location_data

    # Append previous questions as context
    if (i==0):
      message.append(question_data[0])
    else:
      for j in range(i+1):
        message.append(question_data[j])
        if (j<i):
          message.append({'role': 'assistant', 'content': output_msg[j]})

    # Generate response body for API
    response = client.chat.completions.create(
      model=MODEL,
      messages=message
    )

    # Notify if it's too large for current model
    if (response.choices[0].finish_reason == "length"):
      print("ERROR: Too Long")
      return output_msg

    # Obtain results and save it as context for future questions
    output_msg.append(response.choices[0].message.content)

    # Cost Calculations
    input_cost = gpt_pricing["input"] * (response.usage.prompt_tokens / 1000)
    output_cost = gpt_pricing["output"] * (response.usage.completion_tokens / 1000)
    cost = input_cost + output_cost

  print(cost)

  return output_msg



In [33]:
# output = run_gpt(temp, temp_loc, temp_ques)
# output

# ChatCompletion(id='chatcmpl-8LiuiS7iuwFLoZaG86qaBk54NHPDT',
#                choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='B. 14', role='assistant', function_call=None, tool_calls=None))],
#                created=1700188228, model='gpt-3.5-turbo-16k-0613',
#                object='chat.completion',
#                system_fingerprint=None,
#                usage=CompletionUsage(completion_tokens=4, prompt_tokens=7661, total_tokens=7665))


# Main Code Entrypoint


In [34]:
all_uuid = experiments['USER ID'].unique()
len(all_uuid)

66

In [35]:
count = 0
OUTPUT_PATH = userdata.get('OUTPUT_FILE')

for uuid in all_uuid:

  # Generate Output
  ev_exp_df = create_event_experiment_dfs(uuid)
  location_and_instruction = gen_location_and_instruct(ev_exp_df)
  question_data = gen_questions(ev_exp_df)
  output = run_gpt(ev_exp_df, location_and_instruction, question_data)

  # Save output
  count += 1
  df = pd.DataFrame(output, columns=["Response"])
  df.to_csv(f'{OUTPUT_PATH}responseFULLparticipant{count}.csv', index=True)
  break  #   SAFETY NET


RateLimitError: ignored